# Starbucks Capstone Challenge

### Introduction

This data set contains simulated data that mimics customer behavior on the Starbucks rewards mobile app. Once every few days, Starbucks sends out an offer to users of the mobile app. An offer can be merely an advertisement for a drink or an actual offer such as a discount or BOGO (buy one get one free). Some users might not receive any offer during certain weeks. 

Not all users receive the same offer, and that is the challenge to solve with this data set.

Your task is to combine transaction, demographic and offer data to determine which demographic groups respond best to which offer type. This data set is a simplified version of the real Starbucks app because the underlying simulator only has one product whereas Starbucks actually sells dozens of products.

Every offer has a validity period before the offer expires. As an example, a BOGO offer might be valid for only 5 days. You'll see in the data set that informational offers have a validity period even though these ads are merely providing information about a product; for example, if an informational offer has 7 days of validity, you can assume the customer is feeling the influence of the offer for 7 days after receiving the advertisement.

You'll be given transactional data showing user purchases made on the app including the timestamp of purchase and the amount of money spent on a purchase. This transactional data also has a record for each offer that a user receives as well as a record for when a user actually views the offer. There are also records for when a user completes an offer. 

Keep in mind as well that someone using the app might make a purchase through the app without having received an offer or seen an offer.

### Example

To give an example, a user could receive a discount offer buy 10 dollars get 2 off on Monday. The offer is valid for 10 days from receipt. If the customer accumulates at least 10 dollars in purchases during the validity period, the customer completes the offer.

However, there are a few things to watch out for in this data set. Customers do not opt into the offers that they receive; in other words, a user can receive an offer, never actually view the offer, and still complete the offer. For example, a user might receive the "buy 10 dollars get 2 dollars off offer", but the user never opens the offer during the 10 day validity period. The customer spends 15 dollars during those ten days. There will be an offer completion record in the data set; however, the customer was not influenced by the offer because the customer never viewed the offer.

### Cleaning

This makes data cleaning especially important and tricky.

You'll also want to take into account that some demographic groups will make purchases even if they don't receive an offer. From a business perspective, if a customer is going to make a 10 dollar purchase without an offer anyway, you wouldn't want to send a buy 10 dollars get 2 dollars off offer. You'll want to try to assess what a certain demographic group will buy when not receiving any offers.

### Final Advice

Because this is a capstone project, you are free to analyze the data any way you see fit. For example, you could build a machine learning model that predicts how much someone will spend based on demographics and offer type. Or you could build a model that predicts whether or not someone will respond to an offer. Or, you don't need to build a machine learning model at all. You could develop a set of heuristics that determine what offer you should send to each customer (i.e., 75 percent of women customers who were 35 years old responded to offer A vs 40 percent from the same demographic to offer B, so send offer A).

# Data Sets

The data is contained in three files:

* portfolio.json - containing offer ids and meta data about each offer (duration, type, etc.)
* profile.json - demographic data for each customer
* transcript.json - records for transactions, offers received, offers viewed, and offers completed

Here is the schema and explanation of each variable in the files:

**portfolio.json**
* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - minimum required spend to complete an offer
* reward (int) - reward given for completing an offer
* duration (int) - time for offer to be open, in days
* channels (list of strings)

**profile.json**
* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

**transcript.json**
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours since start of test. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record


# Install packages and import libraries

Uncomment and run the following cell to install necessary packages

In [20]:
# TODO: Make a requirements.txt and provide installation instructions
# %conda install seaborn
# %conda update pandas
# %conda install -c conda-forge scikit-learn

In [2]:
import concurrent.futures
import datetime
import json
import math
import os
import random

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import sklearn
from tqdm import tqdm_notebook as tqdm

# Data Preview

Read in the JSON files

In [3]:
portfolio = pd.read_json('data/portfolio.json', orient='records', lines=True)
profile = pd.read_json('data/profile.json', orient='records', lines=True)
transcript = pd.read_json('data/transcript.json', orient='records', lines=True)

Using the Pandas DataFrame info method, we can see an overview of our raw data. These include what columns they are, what data type they were parsed as, the total row counts and how many row of each column are non-nulls.

In [23]:
print(portfolio.info())
print(profile.info())
print(transcript.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   reward      10 non-null     int64 
 1   channels    10 non-null     object
 2   difficulty  10 non-null     int64 
 3   duration    10 non-null     int64 
 4   offer_type  10 non-null     object
 5   id          10 non-null     object
dtypes: int64(3), object(3)
memory usage: 608.0+ bytes
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17000 entries, 0 to 16999
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            14825 non-null  object 
 1   age               17000 non-null  int64  
 2   id                17000 non-null  object 
 3   became_member_on  17000 non-null  int64  
 4   income            14825 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 664.2+ KB
None
<class 'panda

<p>
From that, we can already identify that <i>gender</i> and <i>income</i> columns in <b>profile.json</b> both has 2175 null values. We will need to deal with these null values later.
</p>
<p>
Next, we preview the entirety of <b>portfolio.json</b>, as well as the first 5 rows of <b>profile.json</b> and <b>transcript.json</b>. At the same time, we perform some column renaming.
</p>

In [7]:
portfolio.rename(columns={"id": "offer_id"}, inplace=True)
portfolio

,reward,channels,difficulty,duration,offer_type,offer_id
0,10,"[email, mobile, social]",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd
1,10,"[web, email, mobile, social]",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
2,0,"[web, email, mobile]",0,4,informational,3f207df678b143eea3cee63160fa8bed
3,5,"[web, email, mobile]",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9
4,5,"[web, email]",20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7
5,3,"[web, email, mobile, social]",7,7,discount,2298d6c36e964ae4a3e7e9706d1fb8c2
6,2,"[web, email, mobile, social]",10,10,discount,fafdcd668e3743c1bb461111dcafc2a4
7,0,"[email, mobile, social]",0,3,informational,5a8bc65990b245e5a138643cd4eb9837
8,5,"[web, email, mobile, social]",5,5,bogo,f19421c1d4aa40978ebb69ca19b0e20d
9,2,"[web, email, mobile]",10,7,discount,2906b810c7d4411798c6938adc9daaa5


In [8]:
profile.rename(columns={"id": "person"}, inplace=True)
profile.head()

,gender,age,person,became_member_on,income
0,None,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN
1,F,55,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0
2,None,118,38fe809add3b4fcf9315a9694bb96ff5,20180712,NaN
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0
4,None,118,a03223e636434f42ac4c3df47e8bac43,20170804,NaN


In [5]:
transcript.groupby('event').head(1)

,person,event,value,time
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0
12650,389bc3fa690240e798340f5a15918d5c,offer viewed,{'offer id': 'f19421c1d4aa40978ebb69ca19b0e20d'},0
12654,02c083884c7d45b39cc68e1314fec56c,transaction,{'amount': 0.8300000000000001},0
12658,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,offer completed,{'offer_id': '2906b810c7d4411798c6938adc9daaa5...,0


# Data Transform

<p>
    <b>portfolio.json</b>
</p>
<p>
    No cleaning is necessary. Where <i>duration</i> or <i>reward</i> of rewards is needed, we will join it to other DataFrames keyed on <i>offer_id</i>. That said, it will not be in our inputs as we expect to capture the nuances of user-offer interactions in the offer dummy variables.
</p>
<p>
    <b>transcript.json</b>
</p>
<p>
    Data relevant to each event (e.g. the <i>offer_id</i> for offer-related events, <i>amount</i> transacted for transaction events, <i>reward</i> given for offer completions) are stored in dictionary format in the <i>value</i> column. We extract these values into their own column, effectively splitting <i>value</i> column to give 3 columns: <i>offer_id</i>, <i>amount</i> and <i>reward</i>.
</p>

In [6]:
transcript["offer_id"] = transcript["value"].apply(lambda x: x.get("offer id", x.get("offer_id", np.nan)))
transcript["amount"] = transcript["value"].apply(lambda x: x.get("amount", np.nan))
transcript["reward"] = transcript["value"].apply(lambda x: x.get("reward", np.nan))
transcript.drop(columns="value", inplace=True)
transcript.groupby('event').head(1)

,person,event,time,offer_id,amount,reward
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN,NaN
12650,389bc3fa690240e798340f5a15918d5c,offer viewed,0,f19421c1d4aa40978ebb69ca19b0e20d,NaN,NaN
12654,02c083884c7d45b39cc68e1314fec56c,transaction,0,NaN,0.83,NaN
12658,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,offer completed,0,2906b810c7d4411798c6938adc9daaa5,NaN,2.0


In [28]:
transcript['time'].is_monotonic_increasing

True

<p>
    Each data point in the transactions data represents one event for a single user, and they are in chronological order. This is not meaningful input into a machine learning model.
</p>
<p>
    As we are ultimately interested in users' spending patterns under influences or non-influence of different offers, we transform <b>transcript</b> data such that each row would represent an influence or non-influence period for a user, with transactions aggregated. First, we do this for a random user.
</p>

In [9]:
transcript_by_person = transcript.groupby("person")

#group = random.choice(transcript["person"].unique())
group = '6263d948f09b4cfc87b0b59955f269f1'
df = transcript_by_person.get_group(group)
df

,person,event,time,offer_id,amount,reward
5101,6263d948f09b4cfc87b0b59955f269f1,offer received,0,2906b810c7d4411798c6938adc9daaa5,NaN,NaN
13855,6263d948f09b4cfc87b0b59955f269f1,transaction,0,NaN,8.58,NaN
36098,6263d948f09b4cfc87b0b59955f269f1,transaction,72,NaN,11.88,NaN
36099,6263d948f09b4cfc87b0b59955f269f1,offer completed,72,2906b810c7d4411798c6938adc9daaa5,NaN,2.0
38745,6263d948f09b4cfc87b0b59955f269f1,transaction,84,NaN,13.58,NaN
47001,6263d948f09b4cfc87b0b59955f269f1,transaction,126,NaN,15.78,NaN
58334,6263d948f09b4cfc87b0b59955f269f1,offer received,168,f19421c1d4aa40978ebb69ca19b0e20d,NaN,NaN
67327,6263d948f09b4cfc87b0b59955f269f1,offer viewed,168,f19421c1d4aa40978ebb69ca19b0e20d,NaN,NaN
73269,6263d948f09b4cfc87b0b59955f269f1,transaction,180,NaN,19.94,NaN
73270,6263d948f09b4cfc87b0b59955f269f1,offer completed,180,f19421c1d4aa40978ebb69ca19b0e20d,NaN,5.0


First, we split the DataFrame into offer events, namely <b>offer received</b> / <b>offer viewed</b> / <b>offer completed</b>, and take only <i>time</i> and <i>offer_id</i> columns.

In [10]:
df_received = df[df["event"] == "offer received"][["time", "offer_id"]].rename(columns={"time": "start_time"})
df_viewed = df[df["event"] == "offer viewed"][["offer_id", "time"]].rename(columns={"time": "view_time"})
df_completed = df[df["event"] == "offer completed"][["offer_id", "time"]].rename(columns={"time": "complete_time"})
df_received

,start_time,offer_id
5101,0,2906b810c7d4411798c6938adc9daaa5
58334,168,f19421c1d4aa40978ebb69ca19b0e20d
115999,336,2906b810c7d4411798c6938adc9daaa5
155774,408,9b98b8c7a33c4b65b9aebfe6a799e6d9
206681,504,9b98b8c7a33c4b65b9aebfe6a799e6d9


Secondly, we join the <b>offer received</b> events (which represents the <i>start_time</i> of an offer) with <b>portfolio</b> (which contains the <i>duration</i> of all offers in days) to calculate offer <i>end_time</i>.

In [11]:
offer_periods = df_received.merge(portfolio[["offer_id", "duration"]], on="offer_id", how="left")
offer_periods["end_time"] = offer_periods["start_time"] + offer_periods["duration"] * 24
offer_periods.drop(columns="duration", inplace=True)
offer_periods

,start_time,offer_id,end_time
0,0,2906b810c7d4411798c6938adc9daaa5,168
1,168,f19421c1d4aa40978ebb69ca19b0e20d,288
2,336,2906b810c7d4411798c6938adc9daaa5,504
3,408,9b98b8c7a33c4b65b9aebfe6a799e6d9,576
4,504,9b98b8c7a33c4b65b9aebfe6a799e6d9,672


Thirdly, we seek to find the unique <i>view_time</i> for each of the above offer time blocks. This is done by merging with the <b>offer viewed</b> DataFrame on <i>offer_id</i>. Then, for each time block, we pick the first <i>view_time</i> that falls between the offer <i>start_time</i> and <i>end_time</i> (inclusive), or leave it as null if this offer was never viewed.

In [12]:
offer_periods = offer_periods.merge(df_viewed, on="offer_id", how="left")
offer_periods = offer_periods[
    offer_periods["view_time"].isna() |
    (
        (offer_periods["start_time"] <= offer_periods["view_time"]) &
        (offer_periods["end_time"] >= offer_periods["view_time"])
    )
].groupby(by=["start_time", "offer_id", "end_time"]).first().reset_index()

offer_periods

,start_time,offer_id,end_time,view_time
0,0,2906b810c7d4411798c6938adc9daaa5,168,NaN
1,168,f19421c1d4aa40978ebb69ca19b0e20d,288,168.0
2,336,2906b810c7d4411798c6938adc9daaa5,504,NaN
3,408,9b98b8c7a33c4b65b9aebfe6a799e6d9,576,462.0


Fourthly, we repeat the above step with the <b>offer completed</b> DataFrame to find the unique <i>complete_time</i> for each of the above offer time.

In [13]:
offer_periods = offer_periods.merge(df_completed, on="offer_id", how="left")
offer_periods = offer_periods[
    offer_periods["view_time"].isna() |
    offer_periods["complete_time"].isna() |
    (
        (offer_periods["view_time"] <= offer_periods["complete_time"]) &
        (offer_periods["end_time"] >= offer_periods["complete_time"])
    )
].groupby(by=["start_time", "offer_id", "end_time"]).first().reset_index()

offer_periods

,start_time,offer_id,end_time,view_time,complete_time
0,0,2906b810c7d4411798c6938adc9daaa5,168,NaN,72
1,168,f19421c1d4aa40978ebb69ca19b0e20d,288,168.0,180
2,336,2906b810c7d4411798c6938adc9daaa5,504,NaN,72
3,408,9b98b8c7a33c4b65b9aebfe6a799e6d9,576,462.0,462


These first 4 steps can be wrapped in an <b>offer_periods</b> function.

In [14]:
import inspect

from df_transform import get_offer_periods

print(inspect.getsource(get_offer_periods))

def get_offer_periods(df, portfolio):
    """
    Takes in transaction data for one person and offer data
    Return a DataFrame of offer periods with start, end, view & complete time
    """
    # Split DataFrame by event
    df_received = df[df["event"] == "offer received"][["time", "offer_id"]].rename(columns={"time": "start_time"})
    df_viewed = df[df["event"] == "offer viewed"][["offer_id", "time"]].rename(columns={"time": "view_time"})
    df_completed = df[df["event"] == "offer completed"][["offer_id", "time"]].rename(columns={"time": "complete_time"})
    
    # Calculate offer end time by adding offer duration (in days)
    df = df_received.merge(portfolio[["offer_id", "duration"]], on="offer_id", how="left")
    df["end_time"] = df["start_time"] + df["duration"] * 24
    df.drop(columns="duration", inplace=True)

    # Find unique offer view time (if any) corresponding to each valid offer period
    df = df.merge(df_viewed, on="offer_id", how="left")
    df = df[
        df

<p>
    Fifthly, using the above offer periods, we generate blocks of <i>start_time</i> and <i>end_time</i> representing offer influence periods or uninfluenced periods, from the experiment <i>start_time</i> till the experiment <i>end_time</i>. The logic is:
</p>
<p>
    1. An offer influence period spans from the <i>view_time</i> till <i>end_time</i>, if offer is not completed. Else, it spans from <i>view_time</i> till <i>complete_time</i>. Viewing an offer after completing it does not constitute an offer influence period.<br> (<b>get_offer_periods</b> function)
</p>
<p>
    2. With offer influence periods at hand, whatever timeline gaps from experiment <i>start_time</i> till the experiment <i>end_time</i> are uninfluenced periods, which we fill up with entries with <i>no offer_id</i>.<br> (<b>fill_timeline_gaps</b> generator)
</p>
<p>
    This logic is wrapped in the following <b>get_influence_periods</b> function, which takes in a tuple of a user and transcript DataFrame associated with that user, the overall start and end time, then returns a DataFrame of offer influence periods and uninfluenced periods spanning overall start till end time corresponding to that user:
</p>

In [15]:
from df_transform import get_influence_periods, fill_timeline_gaps

print(inspect.getsource(get_influence_periods))
print(inspect.getsource(fill_timeline_gaps))

def get_influence_periods(grouped, portfolio, start_time, end_time):
    """
    grouped (tuple): ID of person and transcript DataFrame associated with him
    start_time (int): Experiment start time
    end_time (int): Experiment start time
    Returns (pandas.DataFrame): Offer influence periods and uninfluenced periods of the person, 
                                from experiment start till end time
    """
    group, df = grouped
    offer_periods = get_offer_periods(df, portfolio)
    # Offer influence period is from view_time till complete_time (completed) or end_time (not completed)
    offer_periods['start_time'] = offer_periods['view_time']
    offer_periods['end_time'] = offer_periods['end_time'].where(
        offer_periods['complete_time'].isna(), 
        offer_periods['complete_time']
    )
    offer_periods = offer_periods[['start_time', 'end_time', 'offer_id']].dropna()
    # Cuts off at overall end_time
    offer_periods['end_time'] = offer_periods['end_time'].where(o

In [16]:
start_time = transcript["time"].min()
end_time = transcript["time"].max()
print(f"Experiment runs from T={start_time} to T={end_time}")

get_influence_periods(
    (group, transcript_by_person.get_group(group)),
    portfolio,
    start_time,
    end_time,
)

Experiment runs from T=0 to T=714


,start_time,end_time,offer_id,person
0,0.0,168.0,no_offer,6263d948f09b4cfc87b0b59955f269f1
1,168.0,180.0,f19421c1d4aa40978ebb69ca19b0e20d,6263d948f09b4cfc87b0b59955f269f1
1,180.0,462.0,no_offer,6263d948f09b4cfc87b0b59955f269f1
3,462.0,462.0,9b98b8c7a33c4b65b9aebfe6a799e6d9,6263d948f09b4cfc87b0b59955f269f1
2,462.0,714.0,no_offer,6263d948f09b4cfc87b0b59955f269f1


If we haven't already, we use the functions to process the entire transcript data to get data at the granularity of influence periods, each corresponding to an offer an a user. Then we aggregate the transaction amount within each influence periods. Finally, we cache the transformed DataFrame in a <i>inputs_pre.csv</i> under the <b>data/</b> relative directory for use later.

Next, we merge with <b>profile</b> to get all relevant demographic features, namely <i>gender</i>, <i>age</i>, <i>income</i>, and <i>membership_length</i>. Some rows were dropped as the <i>person</i> in <b>transcript</b> does not exist in <b>profile</b> data.

In [37]:
os.listdir("data")

['.ipynb_checkpoints', 'portfolio.json', 'profile.json', 'transcript.json']

In [38]:
%%time
# This transformation may take some time and take up some computing power

if "data/inputs_pre.csv" in os.listdir("data"):
    print("Loading data from existing file...")
    inputs_pre = pd.read_csv("inputs_pre.csv")
else:
    with concurrent.futures.ProcessPoolExecutor() as executor:
        print(f"Transforming data with {executor._max_workers} processes...")
        inputs_pre = pd.concat(
            tqdm(
                executor.map(
                    get_influence_periods,
                    tqdm(transcript_by_person, desc='Mapping for multiprocessing'), 
                    (portfolio for i in range(transcript_by_person.ngroups)),
                    (start_time for i in range(transcript_by_person.ngroups)),
                    (end_time for i in range(transcript_by_person.ngroups)),
                ), desc='Concatenating', total=transcript_by_person.ngroups
            ), axis=0
        )

    del transcript_by_person
    
    transactions_aggregate = inputs_pre.merge(
        transcript.loc[transcript["event"] == "transaction", ["person", "time", "amount"]],
        how="right",
        on=["person"],
    )

    transactions_aggregate = transactions_aggregate[
        (transactions_aggregate["time"] >= transactions_aggregate["start_time"]) &
        (transactions_aggregate["time"] <= transactions_aggregate["end_time"])
    ].groupby(
        by=["person", "offer_id", "start_time", "end_time"]
    )["amount"].sum().reset_index().sort_values(by=["person", "start_time"])

    inputs_pre = inputs_pre.merge(
        transactions_aggregate,
        how='left',
        on=["person", "offer_id", "start_time", "end_time"],
    ).fillna(0)

    del transactions_aggregate
    
    inputs_pre = inputs_pre.merge(
        portfolio[["offer_id", "reward"]],
        how="left",
        on=["offer_id"],
    ).sort_values(by=["person", "start_time"])

    inputs_pre = inputs_pre.merge(
        profile,
        how="left",
        on=["person"],
    )
    
    fp = "data/inputs_pre.csv"
    print(f"Saving data from to {fp}...")
    inputs_pre.to_csv(fp, index=False)

inputs_pre

Transforming data with 8 processes...


<timed exec>:13: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


Mapping for multiprocessing:   0%|          | 0/17000 [00:00<?, ?it/s]

<timed exec>:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


Concatenating:   0%|          | 0/17000 [00:00<?, ?it/s]

Saving data from to data/inputs_pre.csv...
Wall time: 2min 52s


,start_time,end_time,offer_id,person,amount,reward,gender,age,became_member_on,income
0,0.0,192.0,no_offer,0009655768c64bdeb2e877511632db8f,0.00,NaN,M,33,20170421,72000.0
1,192.0,240.0,5a8bc65990b245e5a138643cd4eb9837,0009655768c64bdeb2e877511632db8f,22.16,0.0,M,33,20170421,72000.0
2,240.0,372.0,no_offer,0009655768c64bdeb2e877511632db8f,0.00,NaN,M,33,20170421,72000.0
3,372.0,432.0,3f207df678b143eea3cee63160fa8bed,0009655768c64bdeb2e877511632db8f,8.57,0.0,M,33,20170421,72000.0
4,432.0,714.0,no_offer,0009655768c64bdeb2e877511632db8f,96.87,NaN,M,33,20170421,72000.0
...,...,...,...,...,...,...,...,...,...,...
109823,198.0,354.0,no_offer,ffff82501cea40309d5fdd7edcca4a07,96.37,NaN,F,45,20161125,62000.0
109824,354.0,384.0,2906b810c7d4411798c6938adc9daaa5,ffff82501cea40309d5fdd7edcca4a07,15.57,2.0,F,45,20161125,62000.0
109825,384.0,414.0,no_offer,ffff82501cea40309d5fdd7edcca4a07,33.12,NaN,F,45,20161125,62000.0
109826,414.0,414.0,2906b810c7d4411798c6938adc9daaa5,ffff82501cea40309d5fdd7edcca4a07,17.55,2.0,F,45,20161125,62000.0
